 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [5]:
# Reading whale returns
whale_data = Path("Starter_Code/Resources/whale_returns.csv")
whale_df = pd.read_csv(whale_data, index_col="Date", infer_datetime_format=True, parse_dates=True)
whale_df.sort_index(inplace=True)
whale_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Starter_Code/Resources/whale_returns.csv'

In [3]:
# Count nulls
#Section 3 Day 2 Activity 4

whale_df.isnull().sum()

In [4]:
# Drop nulls
#Section 3 Day 2 Activity 4
whale_df = whale_df.dropna().copy()
whale_df

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_data = Path("../Starter_Code//Resources/algo_returns.csv")
algo_df = pd.read_csv(algo_data, index_col=True, infer_datetime_format=True, parse_dates=True)
algo_df.sort_indes(inplace=True)
algo_df.head()

In [ ]:
# Count nulls

algo_df.isnull().sum()

In [ ]:
# Drop nulls
algo_df.dropna(inplace=True)
algo_df

## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices

sp500_data = Path("../Starter_Code/Resources/sp500_history.csv") 
sp500_df = pd.read_csv(sp500_data, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp500_df.sort_index(inplace=True)
sp500_df.head()

In [ ]:
# Check Data Types

sp500_df.dtypes

In [ ]:
# Fix Data Types
# Find the trick here!

sp500_df["Close"] = sp500_df["Close"].astype(str)
sp500_df["Close"] = sp500_df["Close"].replace({'\$': ''}, regex=True)
sp500_df["Close"] = sp500_df["Close"].astype(float)
sp500_df.dtypes



In [ ]:
# Calculate Daily Returns
sp500_returns = sp500_df.pct_change()
sp500_returns.head()

In [ ]:
# Drop nulls

sp500_returns.dropna()

In [ ]:
# Rename `Close` Column to be specific to this portfolio.

sp500_returns.columns = ["SP 500 Close"]


## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.

combined_returns = pd.concat([whale_df, algo_df, sp500_returns], axis="columns", join="inner")
combined_returns

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios

combined_returns.plot(figsize=(25,15))

#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios

cumulative_returns = (1 + combined_returns).cumprod() - 1

# Plot cumulative returns

cumulative_returns.plot(figsize=(25,15))

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk

cumulative_returns.boxplot(figsize=(25,10))

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios

cumulative_returns.std()

### Determine which portfolios are riskier than the S&P 500

In [ ]:
# Calculate  the daily standard deviation of S&P 500

sp500_returns.std()

# Determine which portfolios are riskier than the S&P 500
## Need help here!! > ****

risk_appraise = cumulative_returns.std()
riskiest_identified = risk_appraise.loc[risk_appraise > 0.144]
riskiest_identified.index

## not sure if needed ****
cumulative_returns[list(riskiest_identified.index)]

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)

annualized_std = cumulative_returns.std() * np.sqrt(252)
annualized_std.sort_values(inplace=True)
annualized_std

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window

rolling_returns_21 = cumulative_returns.rolling(window=21).std()

# Plot the rolling standard deviation

rolling_returns_21.plot(figsize=(25,15))

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation

 correlated_rtns = cumulative_returns.corr()

# Display de correlation matrix

sns.heatmap(correlated_rtns, vmin= -1, vmax=1)


### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [ ]:
# Calculate covariance of a single portfolio

covariance_berk = cumulative_returns["BERKSHIRE HATHAWAY INC"].cov(cumulative_returns["S&P 500 Close"])

# Calculate variance of S&P 500

covariance_SP500 = cumulative_returns["S&P 500 Close"].var()

# Computing beta

beta_SP500_Berk = covariance_berk / covariance_SP500

# Plot beta trend
# ***** May try .plot?

sns.lmplot(x='S&P 500 Close', y='BERKSHIRE HATHAWAY INC', data=cumulative_returns, aspect=1.5, fit_reg=True)



## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [ ]:
# Use `ewm` to calculate the rolling window

rolling_ewm_21 = cumulative_returns.ewm(halflife=21).std()
rolling_ewm_21.plot(figsize=(25,15))

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios

sharp_ratio = (cumulative_returns.mean() * 252) / (cumulative_returns.std() * np.sqrt(252))
sharp_ratio

In [ ]:
# Visualize the sharpe ratios as a bar plot

sharp_ratio.plot(kind=bar)

### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Reading data from 1st stock
# JR choosing to use the provided ptfs - aapl, cost, goog from Resource file

aapl_data = Path("../aapl_historical.csv")
aapl_df = pd.read_csv(aapl_data, index_col="Date", infer_datetime_format=True, parse_dates=True)


In [ ]:
# Reading data from 2nd stock

cost_data = Path("../cost_historical.csv")
cost_df = pd.read_csv(cost_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

In [ ]:
# Reading data from 3rd stock

goog_data = Path("../goog_historical.csv")
goog_df = pd.read_csv(goog_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

In [ ]:
# Combine all stocks in a single DataFrame

combined_ptfs = pd.concat([aapl_df, cost_df, goog_df], axis=1)
combined_ptfs.sort_index(inplace=True)
combined_ptfs.columns = ['Apple', 'Costco', 'Google']
combined_ptfs

In [ ]:
# Reset Date index
# *** did in one line above

In [ ]:
# Reorganize portfolio data by having a column per symbol

## *** did this in above code

In [ ]:
# Calculate daily returns

combined_ptfs = combined_ptfs.pct_change()

# Drop NAs

valid_combined_ptfs = combined_ptfs.dropna()

# Display sample data

valid_combined_ptfs


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return

combined_ptfs_weighted_returns = valid_combined_ptfs.dot(weights)

# Display sample data

combined_ptfs_weighted_returns

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame
## **** double check this

jr_ptf_combined = pd.concat([combined_ptfs_weighted_returns, cumulative_returns], axis="columns", join="inner")
jr_ptf_combined.columns = ["JR PTF", "SOROS", "PAULSON", "TIGER", "BERKSHIRE", "ALGO1", "ALGO2", "SP500"]
jr_ptf_combined

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)

valid_jr_ptf_combined = jr_ptf_combined.dropna()
valid_combined_ptfs.isnull().sum()

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`

jr_combined_std = jr_ptf_combined.std() * np.sqrt(252)
jr_combined_std.sort_values(inplace=True)
jr_combined_std

### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation

jr_combined_std_rolling21 = jr_ptf_combined.rolling(window=21).std()

# Plot rolling standard deviation

jr_combined_std_rolling21.plot(figsize=(25,15))

### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation

jr_combined_correlation = jr_ptf_combined.corr()
sns.heatmap(jr_combined_correlation)

### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [ ]:
# Calculate and plot Beta

cov_jrptf = jr_ptf_combined.rolling(wingdow=60).cov(combined_ptfs["SP500"])
sp500_cov = combined_ptfs["SP500"].var()
jr_combined_beta = cov_jrptf / sp500_cov
jr_combined_beta.plot(figsize=(25,15))

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios

jr_ptf_combined_sharpe = (jr_ptf_combined.mean() * 252) / (jr_ptf_combined.std() * np.sqrt(252))
jr_ptf_combined_sharpe

In [ ]:
# Visualize the sharpe ratios as a bar plot

jr_ptf_combined_sharpe.plot(kind="bar")

### How does your portfolio do?

Write your answer here!